## Source: PubMed
## Retriever: Pubmed Search
## Model: Roberta Large BoolQ

In [11]:
import torch
from sklearn.metrics import classification_report, roc_auc_score

import sys
sys.path.append("../../") # use utils

import utils
import importlib
importlib.reload(utils)

from utils import calc_auc
import pandas as pd
from progressbar import progressbar as pb

In [2]:
from transformers import AutoTokenizer, RobertaForSequenceClassification

model_name = "apugachev/roberta-large-boolq-finetuned"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()
print("OK")

OK


In [3]:
def predict(question, passage, device=device):
    sequence = tokenizer.encode_plus(
        question, 
        passage, 
        return_tensors="pt",
        max_length=tokenizer.model_max_length,
        truncation=True
    )['input_ids']

    logits = model(sequence.to(device))[0]
    probabilities = torch.softmax(logits, dim=1).detach().cpu().tolist()[0]
    proba_yes = probabilities[1]
    
    return proba_yes

In [2]:
init_data = pd.read_csv("../../../data/data_to_process.csv")
print(len(init_data))
init_data.head(3)

113


,data_source,query_id,description,query,label
0,2019,1,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0
1,2019,3,Can acupuncture be effective for people with e...,acupuncture epilepsy,0.0
2,2019,5,Can acupuncture prevent migraines?,acupuncture migraine,1.0


## Keywords

In [4]:
raw_df = pd.read_csv("../../../data/pubmed_search/pubmed_search_abstracts_keywords.csv")

df = (
    raw_df
    .merge(init_data, on=["data_source", "query_id", "label", "query", "description"])
    .dropna()
)
print(len(df))
df.head(1)

645


,query_id,data_source,pubmed_id,description,query,label,abstract
0,1,2019,35285701,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0,Aim: Cranberries ( Vaccinium macrocarpon ) are...


In [6]:
results = []

for _, row in pb(df.iterrows(), max_value=len(df)):
    results.append(predict(row.description, row.abstract))

100% (645 of 645) |######################| Elapsed Time: 0:00:22 Time:  0:00:22


In [7]:
df['prediction'] = results
df.head(1)

,query_id,data_source,pubmed_id,description,query,label,abstract,prediction
0,1,2019,35285701,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0,Aim: Cranberries ( Vaccinium macrocarpon ) are...,0.999816


In [5]:
QUERY_SAVE_PRED_FILE = "predictions/roberta_large_boolq_pubmed_search_preds_keywords.csv"

df.to_csv(QUERY_SAVE_PRED_FILE, index=0)

## Calc Metrics

In [6]:
df = pd.read_csv(QUERY_SAVE_PRED_FILE)
print(len(df))
df.head(2)

645


,query_id,data_source,pubmed_id,description,query,label,abstract,prediction
0,1,2019,35285701,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0,Aim: Cranberries ( Vaccinium macrocarpon ) are...,0.999816
1,1,2019,34205292,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0,Consumption of cranberries is associated with ...,0.997867


In [7]:
agg_types = ["avg", "top1", "norm_linear", "norm_log"]
data_source_types = sorted(df.data_source.unique().tolist()) + ["all"]
data_source_types

['2019', '2021', 'health_belief', 'misbelief', 'all']

In [11]:
df_filled = (
    df
    .merge(init_data, how='outer', on=["query_id", "data_source", "label"])
    .fillna({"prediction": 0.5})
)
print(len(df_filled))
df_filled.tail(5)

678


,query_id,data_source,pubmed_id,description_x,query_x,label,abstract,prediction,description_y,query_y
673,215,misbelief,NaN,NaN,NaN,1.0,NaN,0.5,Does onion kill viruses?,onion kill viruses
674,4_h,health_belief,NaN,NaN,NaN,1.0,NaN,0.5,Does oral sex cause throat cancer?,oral sex causes throat cancer
675,11_h,health_belief,NaN,NaN,NaN,1.0,NaN,0.5,Does nasal polyp cause nasal block?,nasal polyp causes nasal block
676,12_h,health_belief,NaN,NaN,NaN,1.0,NaN,0.5,Does cialis treat enlarged prostrate?,cialis treats enlarged prostrate
677,13_h,health_belief,NaN,NaN,NaN,1.0,NaN,0.5,Does diet cause bad breathe?,diet causes bad breathe


In [9]:
assert 113 == len(df_filled.drop_duplicates(["query_id", "data_source"]))

In [12]:
metrics = {item: [] for item in data_source_types}

for data_source in data_source_types:
    for agg_type in agg_types:
        if data_source == "all":
            metrics[data_source].append(calc_auc(df_filled, agg_type))
        else:
            df_cut = df_filled.query(f"data_source == '{data_source}'")
            metrics[data_source].append(calc_auc(df_cut, agg_type))

metrics_df = pd.DataFrame(metrics, index=agg_types).round(4)
metrics_df

,2019,2021,health_belief,misbelief,all
avg,0.6125,0.8064,0.5000,0.5714,0.6561
top1,0.6540,0.7312,0.3333,0.6071,0.6520
norm_linear,0.6263,0.8064,0.5833,0.5714,0.6614
norm_log,0.6228,0.8064,0.5417,0.5714,0.6595


## Question

In [14]:
raw_df = pd.read_csv("../../../data/pubmed_search/pubmed_search_abstracts_question.csv")

df = (
    raw_df
    .merge(init_data, on=["data_source", "query_id", "label", "description", "query"])
    .dropna()
)
print(len(df))
df.head(1)

507


,query_id,data_source,pubmed_id,description,query,label,abstract
0,1,2019,34444706,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0,Cranberry is a fruit originally from New Engla...


In [15]:
results = []

for _, row in pb(df.iterrows(), max_value=len(df)):
    results.append(predict(row.description, row.abstract))

100% (507 of 507) |######################| Elapsed Time: 0:00:19 Time:  0:00:19


In [16]:
df['prediction'] = results
df.head(1)

,query_id,data_source,pubmed_id,description,query,label,abstract,prediction
0,1,2019,34444706,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0,Cranberry is a fruit originally from New Engla...,0.999866


In [14]:
DESCRIPTION_SAVE_PRED_FILE = "predictions/roberta_large_boolq_pubmed_search_preds_question.csv"

df.to_csv(DESCRIPTION_SAVE_PRED_FILE, index=0)

## Calc Metrics

In [18]:
df = pd.read_csv(DESCRIPTION_SAVE_PRED_FILE)
print(len(df))
df.head(2)

507


,query_id,data_source,pubmed_id,description,query,label,abstract,prediction
0,1,2019,34444706,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0,Cranberry is a fruit originally from New Engla...,0.999866
1,1,2019,33751068,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0,Background: Cranberry proanthocyanidins (c-PAC...,0.999542


In [19]:
df_filled = (
    df
    .merge(init_data, how='outer', on=["query_id", "data_source", "label"])
    .fillna({"prediction": 0.5})
)
print(len(df_filled))
df_filled.tail(5)

550


,query_id,data_source,pubmed_id,description_x,query_x,label,abstract,prediction,description_y,query_y
545,206,misbelief,NaN,NaN,NaN,0.0,NaN,0.5,Does aloe help for a runny nose?,aloe help runny nose
546,211,misbelief,NaN,NaN,NaN,1.0,NaN,0.5,Does licorice root help with cough?,licorice root help cough
547,214,misbelief,NaN,NaN,NaN,1.0,NaN,0.5,Does garlic kill viruses?,garlic kill viruses
548,12_h,health_belief,NaN,NaN,NaN,1.0,NaN,0.5,Does cialis treat enlarged prostrate?,cialis treats enlarged prostrate
549,13_h,health_belief,NaN,NaN,NaN,1.0,NaN,0.5,Does diet cause bad breathe?,diet causes bad breathe


In [17]:
assert 113 == len(df_filled.drop_duplicates(["query_id", "data_source"]))

In [18]:
metrics = {item: [] for item in data_source_types}

for data_source in data_source_types:
    for agg_type in agg_types:
        if data_source == "all":
            metrics[data_source].append(calc_auc(df_filled, agg_type))
        else:
            df_cut = df_filled.query(f"data_source == '{data_source}'")
            metrics[data_source].append(calc_auc(df_cut, agg_type))

metrics_df = pd.DataFrame(metrics, index=agg_types).round(4)
metrics_df

,2019,2021,health_belief,misbelief,all
avg,0.5882,0.6848,0.9167,0.6250,0.6207
top1,0.5848,0.6400,0.7083,0.5893,0.6084
norm_linear,0.5640,0.6896,0.8750,0.6429,0.6220
norm_log,0.5502,0.6896,0.7917,0.6429,0.6113
